# Arabic Dates Classification

In [1]:
import os
import torch
from torch import nn, optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm
import pickle

###   PATHS

In [2]:
DATA_DIR = "/kaggle/input/date-fruit-image-dataset-in-controlled-environment"
MODEL_SAVE_PATH = "arabic_dates_model.pth"
PICKLE_SAVE_PATH = "arabic_dates_classnames.pkl"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

###  DEVICE

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🔧 Using device: {device}")

🔧 Using device: cuda


###   TRANSFORMS

In [4]:
train_transforms = transforms.Compose([
  transforms.Resize((224, 224)),
  transforms.RandomHorizontalFlip(),
  transforms.RandomRotation(10),
  transforms.ToTensor(),
  transforms.Normalize([0.485, 0.456, 0.406],
                       [0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
  transforms.Resize((224, 224)),
  transforms.ToTensor(),
  transforms.Normalize([0.485, 0.456, 0.406],
                       [0.229, 0.224, 0.225])
])

In [5]:
dataset = datasets.ImageFolder(DATA_DIR, transform=train_transforms)
val_size = int(0.2 * len(dataset))
train_size = len(dataset) - val_size
train_data, val_data = random_split(dataset, [train_size, val_size])
val_data.dataset.transform = val_transforms

train_loader = DataLoader(train_data, batch_size=32, shuffle=True, num_workers=2)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False, num_workers=2)

class_names = dataset.classes
num_classes = len(class_names)
print(f"✅ Found {num_classes} classes: {class_names}")

✅ Found 9 classes: ['Ajwa', 'Galaxy', 'Medjool', 'Meneifi', 'Nabtat Ali', 'Rutab', 'Shaishe', 'Sokari', 'Sugaey']


## Model Creation

In [6]:
model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)

for name, param in model.named_parameters():
  param.requires_grad = ("layer4" in name) or ("fc" in name)

model.fc = nn.Sequential(
  nn.Linear(model.fc.in_features, 512),
  nn.ReLU(),
  nn.Dropout(0.4),
  nn.Linear(512, num_classes)
)

model = model.to(device)

criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = optim.Adam([
  {'params': model.layer4.parameters(), 'lr': 1e-5},
  {'params': model.fc.parameters(), 'lr': 1e-4}
])
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)


Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 196MB/s]


## TRAINING FUNCTION

In [7]:
EPOCHS = 10
best_acc = 0.0

for epoch in range(EPOCHS):
  print(f"\n🧠 Epoch {epoch + 1}/{EPOCHS}")
  print("-" * 50)

  # TRAIN
  model.train()
  train_loss, train_correct = 0.0, 0
  for inputs, labels in tqdm(train_loader, desc="Training", leave=False):
    inputs, labels = inputs.to(device), labels.to(device)
    optimizer.zero_grad()
    outputs = model(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    train_loss += loss.item() * inputs.size(0)
    _, preds = torch.max(outputs, 1)
    train_correct += torch.sum(preds == labels.data)

  epoch_train_loss = train_loss / len(train_data)
  epoch_train_acc = train_correct.double() / len(train_data)

  # VALIDATION
  model.eval()
  val_loss, val_correct = 0.0, 0
  with torch.no_grad():
    for inputs, labels in tqdm(val_loader, desc="Validating", leave=False):
      inputs, labels = inputs.to(device), labels.to(device)
      outputs = model(inputs)
      loss = criterion(outputs, labels)
      val_loss += loss.item() * inputs.size(0)
      _, preds = torch.max(outputs, 1)
      val_correct += torch.sum(preds == labels.data)

  epoch_val_loss = val_loss / len(val_data)
  epoch_val_acc = val_correct.double() / len(val_data)

  print(f"📊 Train Loss: {epoch_train_loss:.4f} | Train Acc: {epoch_train_acc:.4f}")
  print(f"🧩 Val Loss:   {epoch_val_loss:.4f} | Val Acc:   {epoch_val_acc:.4f}")

  scheduler.step()

  # SAVE BEST MODEL
  if epoch_val_acc > best_acc:
    best_acc = epoch_val_acc
    torch.save({
      'model_state_dict': model.state_dict(),
      'classes': class_names
    }, MODEL_SAVE_PATH)
    print(f"💾 Saved best model (Val Acc: {best_acc:.4f})")

print(f"\n🎯 Training complete! Best Val Acc: {best_acc:.4f}")
print(f"✅ Model saved to {MODEL_SAVE_PATH}")


🧠 Epoch 1/10
--------------------------------------------------


📊 Train Loss: 2.0838 | Train Acc: 0.3564
🧩 Val Loss:   1.9050 | Val Acc:   0.5589
💾 Saved best model (Val Acc: 0.5589)

🧠 Epoch 2/10
--------------------------------------------------


📊 Train Loss: 1.5613 | Train Acc: 0.6895
🧩 Val Loss:   1.2000 | Val Acc:   0.8610
💾 Saved best model (Val Acc: 0.8610)

🧠 Epoch 3/10
--------------------------------------------------


📊 Train Loss: 0.9659 | Train Acc: 0.9035
🧩 Val Loss:   0.8161 | Val Acc:   0.9184
💾 Saved best model (Val Acc: 0.9184)

🧠 Epoch 4/10
--------------------------------------------------


📊 Train Loss: 0.7275 | Train Acc: 0.9495
🧩 Val Loss:   0.7050 | Val Acc:   0.9305
💾 Saved best model (Val Acc: 0.9305)

🧠 Epoch 5/10
--------------------------------------------------


...

📊 Train Loss: 0.6108 | Train Acc: 0.9834
🧩 Val Loss:   0.6649 | Val Acc:   0.9456
💾 Saved best model (Val Acc: 0.9456)

🧠 Epoch 7/10
--------------------------------------------------


📊 Train Loss: 0.5986 | Train Acc: 0.9872
🧩 Val Loss:   0.6499 | Val Acc:   0.9456

🧠 Epoch 8/10
--------------------------------------------------


📊 Train Loss: 0.5796 | Train Acc: 0.9932
🧩 Val Loss:   0.6450 | Val Acc:   0.9517
💾 Saved best model (Val Acc: 0.9517)

🧠 Epoch 9/10
--------------------------------------------------


📊 Train Loss: 0.5725 | Train Acc: 0.9962
🧩 Val Loss:   0.6399 | Val Acc:   0.9577
💾 Saved best model (Val Acc: 0.9577)

🧠 Epoch 10/10
--------------------------------------------------


📊 Train Loss: 0.5724 | Train Acc: 0.9940
🧩 Val Loss:   0.6383 | Val Acc:   0.9517

🎯 Training complete! Best Val Acc: 0.9577
✅ Model saved to arabic_dates_model.pth


## Model Save

In [8]:
with open(PICKLE_SAVE_PATH, 'wb') as f:
  pickle.dump(class_names, f)
print(f"📦 Saved class names to {PICKLE_SAVE_PATH}")

📦 Saved class names to arabic_dates_classnames.pkl
